In [1]:
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from Custom_dataset import CustomDataset
from resnet1d import ResNet1D

In [2]:
adult_train = pd.read_csv('./dataset/ECG_adult_age_train.csv')
child_train = pd.read_csv('./dataset/ECG_child_age_train.csv')

In [3]:
adult_file_path = './dataset/ECG_adult_numpy_train/'
child_file_path = './dataset/ECG_child_numpy_train/'

In [19]:
adult_train.AGE

0        53
1        68
2        68
3        89
4        79
         ..
34902    19
34903    63
34904    60
34905    60
34906    53
Name: AGE, Length: 34907, dtype: int64

In [4]:
adult_train_data, adult_valid_data = train_test_split(adult_train,shuffle=True, random_state=42, test_size=0.1)
adult_train_data.reset_index(inplace=True,drop=True)
adult_valid_data.reset_index(inplace=True,drop=True)

In [5]:
adult_train_dataset = CustomDataset(adult_file_path,adult_train_data)
adult_valid_dataset = CustomDataset(adult_file_path,adult_valid_data)
child_dataset = CustomDataset(child_file_path,child_train)

In [6]:
train_adult_loader = torch.utils.data.DataLoader(adult_train_dataset, batch_size = 64)
valid_adult_loader = torch.utils.data.DataLoader(adult_valid_dataset, batch_size = 64)

In [7]:
kernel_size = 16
stride = 2
n_block = 48
downsample_gap = 6
increasefilter_gap = 12

In [8]:
model = ResNet1D(
    in_channels=1,
    base_filters=64, # 64 for ResNet1D, 352 for ResNeXt1D
    kernel_size=kernel_size,
    stride=stride, 
    groups=32, 
    n_block=n_block, 
    n_classes=1, 
    downsample_gap=downsample_gap,
    increasefilter_gap=increasefilter_gap, 
    use_do=True)
model.to('cuda')
print('model_ready')
#criterion = torch.nn.MSELoss()
criterion = torch.nn.SmoothL1Loss()

optimizer = torch.optim.Adam(params=model.parameters(),lr = 1e-3)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, mode='min', min_lr = 1e-5, patience=2)
#scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer=optimizer, T_max= 20, eta_min=1e-5)
scaler = torch.cuda.amp.GradScaler()

model_ready


In [10]:
for epochs in range(5):
    print('train_run')
    for batch in tqdm(train_adult_loader):
        inputs = batch[1]['I'].view(-1,1,5000).to('cuda')
        label = torch.tensor(batch[2],dtype=torch.float32).to('cuda')
        with torch.cuda.amp.autocast():
            output = model(torch.tensor(inputs, dtype = torch.float32))
            loss = criterion(output, label)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        optimizer.zero_grad()


    valid_loss = 0
    print('valid_run')
    for batch in tqdm(valid_adult_loader):
        inputs = batch[1]['I'].view(-1,1,5000).to('cuda')
        label = torch.tensor(batch[2],dtype=torch.float32).to('cuda')
        with torch.no_grad():
            output = model(torch.tensor(inputs, dtype = torch.float32))
        loss = criterion(output, label)
        valid_loss += loss
    loss_check = valid_loss/len(valid_adult_loader)
    print(loss_check)
    scheduler.step(loss_check)

train_run


  0%|          | 0/491 [00:00<?, ?it/s]C:\Users\SNUH벤처\AppData\Local\Temp\ipykernel_18364\485264190.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.tensor(batch[2],dtype=torch.float32).to('cuda')
C:\Users\SNUH벤처\AppData\Local\Temp\ipykernel_18364\485264190.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  output = model(torch.tensor(inputs, dtype = torch.float32))
c:\Anaconda\envs\colon\lib\site-packages\torch\nn\modules\loss.py:928: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.smooth_l1_los

valid_run


  0%|          | 0/55 [00:00<?, ?it/s]C:\Users\SNUH벤처\AppData\Local\Temp\ipykernel_18364\485264190.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.tensor(batch[2],dtype=torch.float32).to('cuda')
C:\Users\SNUH벤처\AppData\Local\Temp\ipykernel_18364\485264190.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  output = model(torch.tensor(inputs, dtype = torch.float32))
 98%|█████████▊| 54/55 [00:06<00:00,  7.76it/s]c:\Anaconda\envs\colon\lib\site-packages\torch\nn\modules\loss.py:928: UserWarning: Using a target size (torch.Size([35])) that is different to the input size (torch.Size([35, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure t

tensor(12.5530, device='cuda:0')
train_run


...

tensor(12.4951, device='cuda:0')
train_run


100%|██████████| 491/491 [02:06<00:00,  3.88it/s]


valid_run


100%|██████████| 55/55 [00:06<00:00,  8.16it/s]

tensor(12.4927, device='cuda:0')


In [12]:
pred = []
labels = []
for batch in tqdm(valid_adult_loader):
    inputs = batch[1]['I'].view(-1,1,5000).to('cuda')
    label = torch.tensor(batch[2],dtype=torch.float32).to('cuda')
    with torch.no_grad():
        output = model(torch.tensor(inputs, dtype = torch.float32))
    pred += output.detach().cpu().tolist()
    labels += label.detach().cpu().tolist()

  0%|          | 0/55 [00:00<?, ?it/s]

C:\Users\SNUH벤처\AppData\Local\Temp\ipykernel_18364\616708737.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.tensor(batch[2],dtype=torch.float32).to('cuda')
C:\Users\SNUH벤처\AppData\Local\Temp\ipykernel_18364\616708737.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  output = model(torch.tensor(inputs, dtype = torch.float32))
100%|██████████| 55/55 [00:09<00:00,  6.09it/s]
